In [1]:
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
import word_preprocessing as wp

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
[nltk_data] Downloading package punkt to /Users/binoy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
ml_classes = ['location', 'professor', 'time', 'professor_office_hours',
              'professor_office_location', 'project_details', 'syllabus',
              'exam_details']

processor  =  wp.Word_Processor('../grammar/')
feature_set,class_set,lexicon_size = processor.words_to_vectors()


In [3]:
lstm_units = 128
batch_size = 10
lstm_layers = 1
learning_rate = 0.01

def split_sets(data, classes ,split_point = 1):
    split_length  = int(data.shape[0] * split_point)
    training_features = data[:split_length]
    training_ops = classes[:split_length]
    testing_features = data[split_length:]
    testing_ops = classes[split_length:]
    return training_features,training_ops,testing_features,testing_ops

# currently using all data for training 
trainX,trainY,_,_ = split_sets(feature_set, class_set)
trainX = np.reshape(trainX, newshape=[trainX.shape[0],trainX.shape[1],1])
print(trainX.shape, trainY.shape)

(8328, 55, 1) (8328,)


In [4]:
lstm_net =  tflearn.input_data(shape=[None,lexicon_size,1])
trainY = to_categorical(trainY, nb_classes=len(ml_classes))
#lstm_net = tflearn.embedding(lstm_net)
lstm_net =  tflearn.lstm(lstm_net,128)
lstm_net = tflearn.fully_connected(lstm_net,n_units=len(ml_classes),activation='softmax')
lstm_net = tflearn.regression(lstm_net, optimizer='adam', learning_rate=0.01,
                         loss='categorical_crossentropy')

model = tflearn.DNN(lstm_net, tensorboard_verbose=0)
model.fit(trainX, trainY,show_metric=True,batch_size=10,n_epoch=10)


Training Step: 8329  | total loss: 0.78212 | time: 29.994s
| Adam | epoch: 010 | loss: 0.78212 - acc: 0.6934 -- iter: 8320/8328
Training Step: 8330  | total loss: 0.81264 | time: 30.028s
| Adam | epoch: 010 | loss: 0.81264 - acc: 0.6841 -- iter: 8328/8328
--


In [6]:
test_x = "where is the class"
tr_data = processor.vectorize_input(test_x)
tr_data = np.reshape(tr_data, newshape=[1,lexicon_size,1])
predicted_class = model.predict_label(tr_data)
predicted_values =  model.predict(tr_data)
print(predicted_class)
print(predicted_values)

[[4 3 7 5 0 2 1 6]]
[[  1.04392491e-01   1.22834351e-02   4.03485186e-02   2.13768974e-01
    2.55884111e-01   1.63451731e-01   1.04475555e-04   2.09766299e-01]]
